## Initialize

In [1]:

import requests
import pandas as pd
import os
import sys
from datetime import datetime
from ipywidgets import Dropdown
import json
import base64
import requests

from PyPDF2 import PdfFileReader

sys.path.append(os.path.join(sys.path[0], '..', '..', '..', '..'))
import src.esh_client as esh
from demo.shared.tools import TENANT_SUFFIX, get_base_url, get_root_path

headers = {'Content-Type': 'application/json'}
base_url = get_base_url()



## Choose file folder

In [2]:
folder_path = os.path.join('..')
options = next(os.walk(folder_path))[1]
options.remove('notebooks')
selected = Dropdown(
    options=options,
    description='Select:',
    value = None,
    disabled=False,
)
selected

Dropdown(description='Select:', options=('externDocuments', 'internDocuments'), value=None)

## Take all files from folder

In [16]:
if selected.value:
    payload_list = []
    # Get files path
    current_path = os.path.join(sys.path[0], '..', selected.value)
    # Iterate about the folder with all files
    for filename in os.listdir(current_path):
        # Read PDF File and encode to base64
        with open(os.path.join(current_path, filename), 'rb') as f:  # open in readonly mode
            filename = filename.replace('.pdf', '')

            # encode to base64
            encoded_string = base64.b64encode(f.read())
            encoded_doc = encoded_string.decode('ascii')

            # Get file info
            pdf = PdfFileReader(f)
            info = pdf.getDocumentInfo()
            created_at = str(info.creation_date)
            created_at_string = created_at[0:10]
            changed_at = str(info.modification_date)
            changed_at_string = changed_at[0:10]
            print(filename + ' Created at: ' + created_at_string +
                  ' Changed at: ' + changed_at_string)

            # Append to JSON
            payload = json.dumps({
                'Document': [
                    {
                        'title': filename,
                        'text': encoded_doc,
                        'createdAt': created_at_string,
                        'changedAt': changed_at_string
                    }
                ]
            }, indent=2
            )
            payload_list.append(payload)

else:
    print('No folder selected for PDF files')


Xref table not zero-indexed. ID numbers for objects will be corrected.


AUDI_report Created at: 2022-10-27 Changed at: 2022-11-07
BMW_report Created at: 2022-11-02 Changed at: 2022-11-02
MERCEDES_report Created at: 2022-03-08 Changed at: 2022-03-10
PORSCHE_report Created at: 2021-03-21 Changed at: 2021-03-22
TESLA_report Created at: 2022-10-19 Changed at: 2022-10-19
TOYOTA_report Created at: 2021-08-23 Changed at: 2021-08-23
VW_financial_services_report Created at: 2022-03-23 Changed at: 2022-04-06


## Post data to server

In [17]:
for element in payload_list:
    response = requests.request("POST", f'{base_url}/v1/data/DocResearchDemoExample', headers=headers, data=element)

    print(response.text)

{"Document":[{"id":"ad5a8f37-5f3c-11ed-87dc-88d82e07e733"}]}
{"Document":[{"id":"ae4b4ce0-5f3c-11ed-b6e1-88d82e07e733"}]}
{"Document":[{"id":"ae6faa67-5f3c-11ed-aef4-88d82e07e733"}]}
{"Document":[{"id":"aefb5171-5f3c-11ed-96bf-88d82e07e733"}]}
{"Document":[{"id":"b148d480-5f3c-11ed-8b9a-88d82e07e733"}]}
{"Document":[{"id":"b3c74931-5f3c-11ed-b6a3-88d82e07e733"}]}
{"Document":[{"id":"b3f7d6c3-5f3c-11ed-84c8-88d82e07e733"}]}
